In [ ]:
import pandas as pd
import numpy as np

## Задание 0


Для всех  заданий будем использовать обезличенные транзакционные банковские данные. Для этого считайте в переменные **tr_mcc_codes, tr_types, transactions и gender_train** из одноимённых таблиц из папки [data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L). 

Для таблицы transactions используйте только первые n=1000000 строк. При чтении файлов обратите внимание на разделители внутри каждого из файлов - они могут различаться!

In [ ]:
%%capture
!wget https://www.dropbox.com/s/axkihywa8q118rk/transactions.csv
!wget https://www.dropbox.com/s/xwbkjwik600clsh/tr_mcc_codes.csv
!wget https://www.dropbox.com/s/5deipwkmeial1ba/tr_types.csv
!wget https://www.dropbox.com/s/qc307hdkc9jdxf8/gender_train.csv

transactions = pd.read_csv('transactions.csv', nrows = 1000000)
tr_mcc_codes = pd.read_csv('tr_mcc_codes.csv', sep=';')
gender_train = pd.read_csv('gender_train.csv')
tr_types = pd.read_csv('tr_types.csv',sep=';')


###  Описание данных
#### Таблица ```transactions.csv```
##### Описание
Таблица содержит историю транзакций клиентов банка за один год и три месяца.

##### Формат данных

```
customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
111111,15 01:40:52,1111,1000,-5224,111111
111112,15 15:18:32,3333,2000,-100,11122233
...
```
##### Описание полей

 - ```customer_id``` — идентификатор клиента;
 - ```tr_datetime``` — день и время совершения транзакции (дни нумеруются с начала данных);
 - ```mcc_code``` — mcc-код транзакции;
 - ```tr_type``` — тип транзакции;
 - ```amount``` — сумма транзакции в условных единицах со знаком; ```+``` — начисление средств клиенту (приходная транзакция), ```-``` — списание средств (расходная транзакция);
 - ```term_id``` — идентификатор терминала;


#### Таблица ```gender_train.csv```

##### Описание
Данная таблица содержит информацию по полу для части клиентов, для которых он известен. Для остальных клиентов пол неизвестен.

##### Формат данных
```
customer_id,gender
111111,0
111112,1
...
```

##### Описание полей
 - ```customer_id``` — идентификатор клиента;
 - ```gender``` — пол клиента;

### Таблица ```tr_mcc_codes.csv```

##### Описание
Данная таблица содержит описание mcc-кодов транзакций.

##### Формат данных
```
mcc_code;mcc_description
1000;словесное описание mcc-кода 1000
2000;словесное описание mcc-кода 2000
...
```

##### Описание полей
 - ```mcc_code``` – mcc-код транзакции;
 - ```mcc_description``` — описание mcc-кода транзакции.

#### Таблица ```tr_types.csv```

##### Описание
Данная таблица содержит описание типов транзакций.

##### Формат данных
```
tr_type;tr_description
1000;словесное описание типа транзакции 1000
2000;словесное описание типа транзакции 2000
...
```

##### Описание полей
 - ```tr_type``` – тип транзакции;
 - ```tr_description``` — описание типа транзакции;


Задания 1-4 делаются без использования merge!

## Задание 1



1. Для столбца tr_type датафрейма transactions выберите произвольные 1000 строк с помощью метода sample
2. В полученной на предыдущем этапе подвыборке найдите долю транзакций (стобец tr_description в датасете tr_types), в которой содержится подстрока 'POS' или 'ATM'


In [ ]:

tr_type_pos_atm = (tr_types['tr_type'][( tr_types['tr_description'].str.contains("АТМ")) | ( tr_types['tr_description'].str.contains("POS"))])
count = transactions['tr_type'].sample(1000).isin(tr_type_pos_atm).value_counts()[True]
print(count/1000)


0.637


## Задание 2 


1. Для столбца tr_type датафрейма transactions посчитайте частоту встречаемости всех типов транзакций tr_type в transactions.
2. Выведите топ-10 транзакций по частоте встречаемости (вывести для них tr_description тоже).

In [ ]:
print(transactions['tr_type'].value_counts())
top_ten = transactions['tr_type'].value_counts().sort_values(ascending=False).head(10)
answer = tr_types[tr_types['tr_type'].isin(top_ten.index)]
display(answer)

1010    231117
2010    151166
7070    149006
1110    137658
1030    118975
         ...  
4061         4
4096         4
8146         3
8100         2
1510         2
Name: tr_type, Length: 72, dtype: int64

,tr_type,tr_description
39,7010,Взнос наличных через АТМ (в своем тер.банке)
47,7030,Перевод на карту (с карты) через АТМ (в предел...
59,7070,Перевод на карту (с карты) через Мобильный бан...
60,7071,Перевод на карту (с карты) через Мобильный бан...
98,1010,Покупка. POS ТУ СБ РФ
99,1030,Оплата услуги. Банкоматы СБ РФ
100,1100,Покупка. ТУ Россия
101,1110,Покупка. POS ТУ Россия
106,2010,Выдача наличных в АТМ Сбербанк России
129,2370,Списание с карты на карту по операции <перевод...


## Задание 3
1. В датафрейме transactions найдите клиента с максимальной суммой приходов на карту
2. В датафрейме transactions найдите клиента с максимальной суммой расходов по карте
3. Найдите модуль разницы для этих клиентов между суммой расходов и суммой приходов

In [ ]:
max_earning = transactions[transactions['amount']>0].groupby(['customer_id']).sum().nlargest(1,'amount')
min_earning = transactions[transactions['amount']<=0].groupby(['customer_id']).sum().nsmallest(1,'amount')
print(max_earning['amount'].to_string())
print(min_earning['amount'].to_string())
print((max_earning['amount']+min_earning['amount']).abs().to_string())
max_earning_customer = max_earning.index.tolist()
min_earning_customer = min_earning.index.tolist()


customer_id
70780820    1.248115e+09
customer_id
70780820   -1.249952e+09
customer_id
70780820    1.837318e+06


## Задание 4
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций из топ 10 из задания 2
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций для клиентов из задания 3

In [ ]:
top_ten = transactions['tr_type'].value_counts().sort_values(ascending=False).head(10)
tr_types = top_ten.index
ten_transactions =transactions[transactions['tr_type'].isin(tr_types)]
mean = ten_transactions.groupby(['tr_type'])['amount'].mean()
median = ten_transactions.groupby(['tr_type'])['amount'].median()
mean_median = pd.concat([mean,median],axis = 1, keys=['Mean', 'Median'])
print(mean_median)

                  Mean     Median
tr_type                          
1010     -19784.748640   -7411.52
1030      -5320.980222   -2245.92
1100     -44061.827262  -10188.26
1110     -32119.330371  -11207.57
2010    -136077.629325  -44918.32
2370    -205418.249032  -44918.32
7010     276391.789596  112295.79
7030      86104.332909   13951.52
7070      65569.831700   11319.42
7071      66806.826623    3593.47


In [ ]:
max_client_transactions = transactions[transactions['customer_id'].isin(max_earning_customer)]
mean = max_client_transactions.groupby(['tr_type'])['amount'].mean()
median = max_client_transactions.groupby(['tr_type'])['amount'].median()
mean_median = pd.concat([mean,median],axis = 1, keys=['Mean', 'Median'])
print(mean_median)
min_client_transactions = transactions[transactions['customer_id'].isin(min_earning_customer)]
mean = min_client_transactions.groupby(['tr_type'])['amount'].mean()
median = min_client_transactions.groupby(['tr_type'])['amount'].median()
mean_median = pd.concat([mean,median],axis = 1, keys=['Mean', 'Median'])
print(mean_median)

                 Mean       Median
tr_type                           
2010    -2.941056e+06 -3368873.660
2330    -2.382398e+06 -2245915.770
2370    -2.218505e+06 -2245915.770
6110     1.756293e+04    10028.010
7020     1.465129e+04    10319.985
7021     2.850067e+04    28500.670
7030     1.406196e+04     8848.910
7031     1.921036e+04    12992.620
7034     1.430648e+04    14306.480
7040     2.153833e+04    14396.320
7070     1.384821e+04     8714.150
7071     1.860057e+04    12543.440
7074     2.281159e+04     9387.930
                 Mean       Median
tr_type                           
2010    -2.941056e+06 -3368873.660
2330    -2.382398e+06 -2245915.770
2370    -2.218505e+06 -2245915.770
6110     1.756293e+04    10028.010
7020     1.465129e+04    10319.985
7021     2.850067e+04    28500.670
7030     1.406196e+04     8848.910
7031     1.921036e+04    12992.620
7034     1.430648e+04    14306.480
7040     2.153833e+04    14396.320
7070     1.384821e+04     8714.150
7071     1.860057e+0

## Подготовка для заданий 5-8

*Из заданий 5-8 нужно выполнить минимум два любых*

Соедините transactions с всеми остальными таблицами (tr_mcc_codes, tr_types, gender_train). Причём с gender_train необходимо смёрджиться с помощью left join, а с оставшимися датафреймами - через inner.
После получения результата таблицы gender_train, tr_types, tr_mcc_codes можно удалить. В результате соединения датафреймов должно получиться 999584 строки.

In [ ]:
transactions = pd.merge(transactions, gender_train, how='left')
transactions = pd.merge(transactions, tr_mcc_codes, how='inner')
transactions = pd.merge(transactions, tr_types, how='inner')
transactions.shape

(999584, 9)

## Задание 5

1. Определите модуль разницы между средними тратами женщин и мужчин (трата - отрицательное значение amount).
2. Определите модуль разницы между средними поступлениями у мужчин и женщин

Обратите внимание, что для вычисления модуля разности точных знаний о том, 
какой класc относится к мужчинам, а какой - к женщинам, не требуется.

In [ ]:
negative_transactions = transactions[transactions['amount']<0].groupby('gender')['amount'].mean()
display(abs(negative_transactions[0]-negative_transactions[1]))
positive_transactions = transactions[transactions['amount']>0].groupby('gender')['amount'].mean()
display(abs(positive_transactions[0]-positive_transactions[1]))

32718.05492021877

63366.57104801515

## Задание 6

1. По всем типам транзакций рассчитайте максимальную сумму прихода на карту (из строго положительных сумм по столбцу amount) отдельно для мужчин и женщин (назовите ее "max_income"). Оставьте по 10 типов транзакций для мужчин и для женщин, наименьших среди всех типов транзакций по полученным значениям "max_income". 
2. Выделите среди них те типы транзакций, которые встречаются одновременно и у мужчин, и у женщин

In [ ]:
tr_types_min = transactions[(transactions['amount']>0) & (transactions['gender'] == 0)].groupby('tr_type')['amount'].max()
antitop_ten_0 = tr_types_min.sort_values(ascending=True).head(10)
print(antitop_ten_0)
tr_types_min_1 = transactions[(transactions['amount']>0) & (transactions['gender'] == 1)].groupby('tr_type')['amount'].max()
antitop_ten_1 = tr_types_min_1.sort_values(ascending=True).head(10)
print(antitop_ten_1)

tr_type
4051     1122.96
4110     2245.92
4210     2245.92
2370     2335.75
4100     5041.86
7075     6737.75
2110    22459.16
1010    24839.83
2010    44918.32
2210    68315.82
Name: amount, dtype: float64
tr_type
4110      2245.92
2020      2245.92
1010     28803.87
8100     64682.37
6000     79280.83
7041     87590.72
2010     89836.63
2110    112295.79
2370    113194.15
7015    121279.45
Name: amount, dtype: float64


In [ ]:
print(pd.concat([antitop_ten_0,antitop_ten_1], axis=1).dropna())

           amount     amount
tr_type                     
1010     24839.83   28803.87
2010     44918.32   89836.63
2110     22459.16  112295.79
2370      2335.75  113194.15
4110      2245.92    2245.92


## Задание 7

1. Найдите суммы затрат по каждой категории (mcc) для мужчин и для женщин. 
2. Найдите топ 10 категорий с самыми большими относительными модулями разности в тратах для разных полов (в ответе должны присутствовать описания mcc кодов).

In [ ]:
category_spendings_0 = transactions[(transactions['amount']<0) & (transactions['gender'] == 0)].groupby('mcc_code')['amount'].sum()
display(category_spendings_0)
category_spendings_1 = transactions[(transactions['amount']<0) & (transactions['gender'] == 1)].groupby('mcc_code')['amount'].sum()
display(category_spendings_1)

mcc_code
742      -386207.68
1711     -674897.69
1799     -261626.73
2741      -20746.42
3000   -74431645.27
           ...     
8699     -521107.48
8999   -19432057.92
9222     -361143.24
9311     -406197.45
9399    -1481389.65
Name: amount, Length: 174, dtype: float64

mcc_code
742      -132277.93
1711     -206264.90
1731     -121542.67
1799    -1129257.68
2741     -221290.08
           ...     
8999   -28643346.63
9222     -988205.17
9311     -373271.22
9399    -1810820.67
9402       -4581.67
Name: amount, Length: 175, dtype: float64

In [ ]:
mcc_descripto = abs((category_spendings_1-category_spendings_0)).sort_values(ascending=False).head(10)
tr_mcc_codes[tr_mcc_codes["mcc_code"].isin(mcc_descripto.index)]

,mcc_code,mcc_description
22,4829,Денежные переводы
62,5511,"Легковой и грузовой транспорт — продажа, серви..."
64,5533,Автозапчасти и аксессуары
65,5541,Станции техобслуживания
75,5661,Обувные магазины
90,5812,"Места общественного питания, рестораны"
116,5977,Магазины косметики
123,6010,Финансовые институты — снятие наличности вручную
124,6011,Финансовые институты — снятие наличности автом...
162,7995,Транзакции по азартным играм


## Задание 8

1. Из поля tr_datetime выделите час tr_hour, в который произошла транзакция, как первые 2 цифры до ":". (\**)
2. Посчитайте количество транзакций с amount<0 в ночное время для мужчин и женщин. Ночное время - это примерно 00-06 часов. 

In [ ]:
transactions[["tr_day", "tr_time"]] = transactions["tr_datetime"].str.split(" ", expand=True)
transactions[["tr_hour", "tr_minute", "tr_second"]] = transactions["tr_time"].str.split(":", expand=True)
display(transactions["tr_day"])


0           0
1           6
2           8
3          11
4          11
         ... 
999579     18
999580    206
999581     90
999582    104
999583    105
Name: tr_day, Length: 999584, dtype: object

In [ ]:
spendings = transactions[(transactions['amount']<0)]
spendings = spendings[(transactions['tr_hour'].astype(int)<6)]
spendings_gender1 = len(spendings[(transactions["gender"]==1)])
spendings_gender0 = len(spendings[(transactions["gender"]==0)])
print("gender1")
print(spendings_gender1)
print("gender0")
print(spendings_gender0)

gender1
43017
gender0
38827


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


Доп
По каждой категории найдите сумму трат в ночное время и в остальное время. Найдите соотношение одного к другому. Выведите список 10 категорий с описаниями, упорядоченных по убываний отношения ночных трат к остальным тратам.  

In [ ]:
spendings = transactions[(transactions['amount']<0)]
night_spendings = spendings[(transactions['tr_hour'].astype(int)<6)]
night_spendings_sum = night_spendings.groupby('tr_type')['amount'].sum()
day_spendings = spendings[(transactions['tr_hour'].astype(int)>=6)]
day_spendings_sum = day_spendings.groupby('tr_type')['amount'].sum()
pd.options.display.float_format='{:.5f}'.format
display(night_spendings_sum)
display(day_spendings_sum)
display((night_spendings_sum/day_spendings_sum).dropna())
ans = (night_spendings_sum/day_spendings_sum).nlargest(10)
display(ans)
answer = tr_types[tr_types['tr_type'].isin(ans.index)]
display(answer.merge(ans,left_on='tr_type', right_on='tr_type').sort_values(by='amount',ascending=False))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


tr_type
1010    -143078520.21000
1030     -52853418.99000
1100    -550511677.98000
1110   -3023498923.89989
1200    -355096730.95001
1210    -411345688.52000
1410      -3548820.27000
1510        -22396.05000
2010   -1141154547.06002
2011     -31447498.45000
2020     -52289299.85000
2100      -6510665.22000
2110     -78531754.29000
2200      -4101425.34000
2210     -87594549.64000
2320       -830073.17000
2330    -125025204.06000
2331     -29380733.26000
2370    -628724707.60001
2371     -89568297.50000
2440      -1554773.61000
4010      -1402742.54000
4011        -72900.99000
4020        -16445.27000
4041         -3368.88000
4051       -282577.75000
4071       -738468.95000
4090         -8422.18000
4100       -121320.74000
4110      -2381804.99000
4200        -43748.19000
4210      -1040809.42000
4500        -52779.03000
7020      -1181576.29000
7030       -120830.26000
7070       -593011.62000
Name: amount, dtype: float64

tr_type
1000        -689054.37000
1010    -4429580080.82966
1030     -580210202.89997
1100     -136912675.92000
1110    -1409415843.38001
1200      -17222818.39000
1210     -102373624.57000
1310       -2566551.70000
1410        -667047.54000
2010   -19430149062.26497
2011    -1005145801.34002
2020    -1110294933.92000
2021      -63402340.62000
2100        -453983.80000
2110      -29995328.38000
2200         -14622.48000
2210      -90125091.33000
2320      -58522431.77000
2330    -2266628084.88000
2331     -770511079.57001
2340      -28312124.07000
2341        -239639.22000
2370    -9608740950.59873
2371    -1914085773.74001
2440      -45105809.40000
4010      -26525281.10000
4011       -2612007.57000
4020        -417108.75000
4021         -56512.83000
4031        -268760.44000
4041        -338387.20000
4051       -6815299.07000
4061          -4042.63000
4071      -15819926.30000
4090        -255394.23000
4100         -15767.22000
4110        -853067.41000
4200          -3368.87000
4210

tr_type
1010     0.03230
1030     0.09109
1100     4.02090
1110     2.14521
1200    20.61781
1210     4.01808
1410     5.32019
2010     0.05873
2011     0.03129
2020     0.04709
2100    14.34118
2110     2.61813
2200   280.48767
2210     0.97192
2320     0.01418
2330     0.05516
2331     0.03813
2370     0.06543
2371     0.04679
2440     0.03447
4010     0.05288
4011     0.02791
4020     0.03943
4041     0.00996
4051     0.04146
4071     0.04668
4090     0.03298
4100     7.69449
4110     2.79205
4200    12.98601
4210     1.08939
4500     0.07281
Name: amount, dtype: float64

tr_type
2200   280.48767
1200    20.61781
2100    14.34118
4200    12.98601
4100     7.69449
1410     5.32019
1100     4.02090
1210     4.01808
4110     2.79205
2110     2.61813
Name: amount, dtype: float64

,tr_type,tr_description,amount
9,2200,Наличные. Зарубеж. банк,280.48767
5,1200,Покупка. Зарубеж. ТУ,20.61781
7,2100,Наличные. Россия,14.34118
2,4200,Плата за получение наличных. Зарубеж.,12.98601
0,4100,Плата за получение наличных. Россия,7.69449
3,1410,н/д,5.32019
4,1100,Покупка. ТУ Россия,4.02090
6,1210,Покупка. POS Зарубеж. ТУ,4.01808
1,4110,Плата за получение наличных в АТМ. Россия,2.79205
8,2110,Наличные. АТМ Россия,2.61813
